**Import Libary**
========================
Pada tahapan ini, Kita akan mengimpor beberapa pustaka library python yang akan dibutuhkan analisis data dalam pembagunan machine learning.
---------------------------------------------------------------------

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from imblearn.over_sampling import SMOTE

**Membuat Dataset dari Hasil Clustering**
===============================================

Membuat hasil clustering dari hasil csvke dalam variable dataframe
--------------------------------------------------------------------

In [2]:
url = "https://raw.githubusercontent.com/Sinestesiaaa/BMLUP_TehranHouse/refs/heads/main/Dataset/cluster_summary.csv"
df = pd.read_csv(url)
df.info()
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2863 entries, 0 to 2862
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Area        2863 non-null   float64
 1   Address     2863 non-null   object 
 2   Price       2863 non-null   float64
 3   Price(USD)  2863 non-null   float64
 4   Cluster     2863 non-null   int64  
 5   Parking     2863 non-null   bool   
 6   Warehouse   2863 non-null   bool   
 7   Elevator    2863 non-null   bool   
 8   Room        2863 non-null   int64  
dtypes: bool(3), float64(3), int64(2), object(1)
memory usage: 142.7+ KB


,Area,Address,Price,Price(USD),Cluster,Parking,Warehouse,Elevator,Room
0,63.0,Shahran,1.850000e+09,61666.67,0,True,True,True,1
1,60.0,Shahran,1.850000e+09,61666.67,0,True,True,True,1
2,79.0,Pardis,5.500000e+08,18333.33,0,True,True,True,2
3,95.0,Shahrake Qods,9.025000e+08,30083.33,0,True,True,True,2
4,123.0,Shahrake Gharb,7.000000e+09,233333.33,0,True,True,True,2


In [3]:
df_normalized = df.copy()
scaler = StandardScaler()
columns_to_scale = ['Area', 'Price', 'Price(USD)']
df_normalized[columns_to_scale] = scaler.fit_transform(df_normalized[columns_to_scale])
df_normalized.head()

,Area,Address,Price,Price(USD),Cluster,Parking,Warehouse,Elevator,Room
0,-0.854429,Shahran,-0.475643,-0.475643,0,True,True,True,1
1,-0.965237,Shahran,-0.475643,-0.475643,0,True,True,True,1
2,-0.263454,Pardis,-1.099127,-1.099127,0,True,True,True,2
3,0.327521,Shahrake Qods,-0.930067,-0.930067,0,True,True,True,2
4,1.361727,Shahrake Gharb,1.994315,1.994315,0,True,True,True,2


In [4]:
min_max_scaler = MinMaxScaler()
df_normalized['Room_scaled'] = min_max_scaler.fit_transform(df[['Room']])
df_normalized.head()

,Area,Address,Price,Price(USD),Cluster,Parking,Warehouse,Elevator,Room,Room_scaled
0,-0.854429,Shahran,-0.475643,-0.475643,0,True,True,True,1,0.2
1,-0.965237,Shahran,-0.475643,-0.475643,0,True,True,True,1,0.2
2,-0.263454,Pardis,-1.099127,-1.099127,0,True,True,True,2,0.4
3,0.327521,Shahrake Qods,-0.930067,-0.930067,0,True,True,True,2,0.4
4,1.361727,Shahrake Gharb,1.994315,1.994315,0,True,True,True,2,0.4


In [5]:
df_normalized.drop(columns=['Room'], inplace=True)

df_normalized.head()

,Area,Address,Price,Price(USD),Cluster,Parking,Warehouse,Elevator,Room_scaled
0,-0.854429,Shahran,-0.475643,-0.475643,0,True,True,True,0.2
1,-0.965237,Shahran,-0.475643,-0.475643,0,True,True,True,0.2
2,-0.263454,Pardis,-1.099127,-1.099127,0,True,True,True,0.4
3,0.327521,Shahrake Qods,-0.930067,-0.930067,0,True,True,True,0.4
4,1.361727,Shahrake Gharb,1.994315,1.994315,0,True,True,True,0.4


In [6]:
encoder = OneHotEncoder(drop='first')
encoded_features = encoder.fit_transform(df[['Parking', 'Warehouse', 'Elevator']])

In [7]:
encoded_df = pd.DataFrame(encoded_features.toarray(), columns=encoder.get_feature_names_out(['Parking', 'Warehouse', 'Elevator']))

df_classification = pd.concat([df_normalized.drop(columns=['Parking', 'Warehouse', 'Elevator']), encoded_df], axis=1)

df_classification.head()

,Area,Address,Price,Price(USD),Cluster,Room_scaled,Parking_True,Warehouse_True,Elevator_True
0,-0.854429,Shahran,-0.475643,-0.475643,0,0.2,1.0,1.0,1.0
1,-0.965237,Shahran,-0.475643,-0.475643,0,0.2,1.0,1.0,1.0
2,-0.263454,Pardis,-1.099127,-1.099127,0,0.4,1.0,1.0,1.0
3,0.327521,Shahrake Qods,-0.930067,-0.930067,0,0.4,1.0,1.0,1.0
4,1.361727,Shahrake Gharb,1.994315,1.994315,0,0.4,1.0,1.0,1.0


In [8]:
df_classification.replace("", np.nan, inplace=True)
df_classification.dropna(inplace=True)

# Cek jumlah baris yang tersisa
print(f"Jumlah baris yang tersisa setelah menghapus missing values: {df.shape[0]}")

# Cek jumlah missing values setelah penghapusan
print("Jumlah missing values setelah penghapusan baris yang memiliki NaN di salah satu kolom:")
print(df_classification.isnull().sum())

Jumlah baris yang tersisa setelah menghapus missing values: 2863
Jumlah missing values setelah penghapusan baris yang memiliki NaN di salah satu kolom:
Area              0
Address           0
Price             0
Price(USD)        0
Cluster           0
Room_scaled       0
Parking_True      0
Warehouse_True    0
Elevator_True     0
dtype: int64


**Data Splitting**
===================

Pada tahap ini kita akan membuat data latih (Traning Set) dan data uji (test set)
-------------------------------

In [9]:
X = df_classification.drop(columns=['Cluster', 'Address','Price(USD)'])
y = df_classification['Cluster']

X.head()

,Area,Price,Room_scaled,Parking_True,Warehouse_True,Elevator_True
0,-0.854429,-0.475643,0.2,1.0,1.0,1.0
1,-0.965237,-0.475643,0.2,1.0,1.0,1.0
2,-0.263454,-1.099127,0.4,1.0,1.0,1.0
3,0.327521,-0.930067,0.4,1.0,1.0,1.0
4,1.361727,1.994315,0.4,1.0,1.0,1.0


In [10]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

splits = {
    "80-20": train_test_split(X, y, test_size=0.2, random_state=42, stratify=y),
    "70-30": train_test_split(X, y, test_size=0.3, random_state=42, stratify=y),
    "60-40": train_test_split(X, y, test_size=0.4, random_state=42, stratify=y),
}

**Membangun Model Klasifikasi**
===============================



1.   Model Klasifikasi
-----------------------------

Memilih alogritma yang sesuai untuk klasifikasi untuk melatih model menggunakan data latih

Berikut rekomendasi yang saya gunakan dalam tahapan ini
* Algoritma yang sesuai seperti logistik Regression, Decision Tree, Radom forest dan K-Nearest Neighbors (KNN)



In [11]:
models = {
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
}

enjelasan alogritma yang kita gunakan

**Mengevaluasi Model Klasifikasi**
===================================

Berikut adalah rekomendasi beserta tahapan-tahapan nya.
----------------------------------------------------------
1. Melakukan prediksi menggunakan data uji
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [12]:
results = []

for split_name, (X_train, X_test, y_train, y_test) in splits.items():
    for model_name, model in models.items():

        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None


        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average="weighted")
        recall = recall_score(y_test, y_pred, average="weighted")
        f1 = f1_score(y_test, y_pred, average="weighted")


        results.append({
            "Split": split_name,
            "Model": model_name,
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1-Score": f1,
        })

results_df = pd.DataFrame(results)
print(results_df)

   Split          Model  Accuracy  Precision    Recall  F1-Score
0  80-20  Decision Tree  0.998255   0.998295  0.998255  0.998258
1  80-20  Random Forest  0.998255   0.998295  0.998255  0.998258
2  70-30  Decision Tree  0.998836   0.998854  0.998836  0.998837
3  70-30  Random Forest  0.998836   0.998854  0.998836  0.998837
4  60-40  Decision Tree  0.998255   0.998275  0.998255  0.998258
5  60-40  Random Forest  0.998255   0.998275  0.998255  0.998258


1. Akurasi (Accuracy)
  - Kedua model memiliki akurasi yang sangat tinggi di semua skenario, berkisar antara 99.82% hingga 99.88%.

  - Akurasi tertinggi diperoleh pada split 70-30, baik untuk Decision Tree maupun Random Forest (0.998836).

  - Tidak ada perbedaan akurasi signifikan antara Decision Tree dan Random Forest, menunjukkan bahwa keduanya bekerja dengan sangat baik pada dataset ini.
  
2. Pengaruh Pembagian Data (Split Ratio)
  - Split 70-30 memberikan hasil terbaik di semua metrik dibandingkan split lainnya (80-20 dan 60-40).

  - Split 60-40 sedikit lebih rendah dibandingkan 70-30, tetapi tetap memiliki performa yang hampir sama.

  - Split 80-20 dan 60-40 memiliki nilai identik, yang bisa jadi disebabkan oleh pola data yang serupa dalam kedua skenario.

3. Precision, Recall, dan F1-Score
    - Semua metrik lainnya juga sangat tinggi (> 99.8%) di semua skenario.

    - Random Forest dan Decision Tree memiliki nilai yang identik dalam setiap pembagian data, yang menunjukkan bahwa dataset memiliki pola yang dapat dipahami dengan baik oleh kedua algoritma.
  
4. Kesimpulan dan Rekomendasi
  - Keunggulan Random Forest vs. Decision Tree:

    - Tidak ada perbedaan signifikan antara Random Forest dan Decision Tree pada dataset ini.

    - Biasanya, Random Forest lebih unggul dalam generalisasi, tetapi dalam kasus ini, Decision Tree sudah cukup baik dengan performa yang hampir identik.

  - Pengaruh Split Data:

    - Split 70-30 adalah pilihan terbaik, karena memberikan akurasi tertinggi (0.998836).

    - Split 80-20 dan 60-40 memiliki hasil yang sama, menunjukkan bahwa ukuran dataset dalam skenario ini tidak terlalu berpengaruh pada performa model.

**Tuning Model Klasifikasi**
===============================

Gunakan GridSearch, RandomizedSearchCV dan metode untuk mencari kombinasi hypermeter yang terbaik
-------------------------------------------------------------------------

In [13]:
dt_params = {
    "criterion": ["gini", "entropy"],
    "max_depth": [5, 10, 15, 20, None],
    "min_samples_split": [2, 5, 10, 20],
    "min_samples_leaf": [1, 2, 5, 10]
}

# Hyperparameter untuk Random Forest
rf_params = {
    "n_estimators": [50, 100, 200, 300],
    "criterion": ["gini", "entropy"],
    "max_depth": [5, 10, 20, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 5]
}

**Mengevaluasi Model setelah berhasil di Tunning**
===================================================

Berikut adalah tahapan-tahapan nya dalam melakukan evaluasi
-----------------------------------------------------------
* Gunakan model hyperparameter terbaik
* Hitung ulang metrik untuk melihat performa.

In [14]:
models = {
    "Decision Tree": (DecisionTreeClassifier(), dt_params),
    "Random Forest": (RandomForestClassifier(), rf_params),
}

results = []

for split_name, (X_train, X_test, y_train, y_test) in splits.items():
    for model_name, (model, params) in models.items():
        # Gunakan RandomizedSearchCV
        search = RandomizedSearchCV(model, params, n_iter=10, cv=5, scoring="accuracy", n_jobs=-1, random_state=42)
        search.fit(X_train, y_train)

        best_model = search.best_estimator_
        best_params = search.best_params_

        y_pred = best_model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average="weighted")
        recall = recall_score(y_test, y_pred, average="weighted")
        f1 = f1_score(y_test, y_pred, average="weighted")

        results.append({
            "Split": split_name,
            "Model": model_name,
            "Best Params": best_params,
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1-Score": f1,
        })

results_df_tuning = pd.DataFrame(results)

pd.set_option("display.max_colwidth", None)
pd.set_option("display.colheader_justify", "center")

print(results_df_tuning.to_string(index=False))


Split     Model                                                     Best Params                                                  Accuracy  Precision  Recall   F1-Score
80-20 Decision Tree                      {'min_samples_split': 20, 'min_samples_leaf': 2, 'max_depth': 20, 'criterion': 'gini'}  0.998255  0.998295  0.998255  0.998258
80-20 Random Forest  {'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': 5, 'criterion': 'gini'}  0.998255  0.998295  0.998255  0.998258
70-30 Decision Tree                      {'min_samples_split': 20, 'min_samples_leaf': 2, 'max_depth': 20, 'criterion': 'gini'}  0.998836  0.998854  0.998836  0.998837
70-30 Random Forest {'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': 10, 'criterion': 'gini'}  0.998836  0.998854  0.998836  0.998837
60-40 Decision Tree                      {'min_samples_split': 20, 'min_samples_leaf': 2, 'max_depth': 20, 'criterion': 'gini'}  0.998255  0.998275  0.998255  0

**Menganalisis Hasil Model Klasifikasi**
=======================================

Berikut beserta tahapan-tahapannya
---------------------------------
* Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.
* Bandingkan hasil evaluasi sebelum dan setelah tuning
* Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?


In [15]:
df_comparison = results_df.merge(
    results_df_tuning[["Split", "Model", "Accuracy", "Precision", "Recall", "F1-Score"]],
    on=["Split", "Model"],
    suffixes=(" (Sebelum)", " (Sesudah)")
)

pd.set_option("display.max_columns", None)
pd.set_option("display.colheader_justify", "center")

print(df_comparison.to_string(index=False))

Split     Model      Accuracy (Sebelum)  Precision (Sebelum)  Recall (Sebelum)  F1-Score (Sebelum)  Accuracy (Sesudah)  Precision (Sesudah)  Recall (Sesudah)  F1-Score (Sesudah)
80-20 Decision Tree       0.998255            0.998295            0.998255           0.998258            0.998255            0.998295            0.998255           0.998258     
80-20 Random Forest       0.998255            0.998295            0.998255           0.998258            0.998255            0.998295            0.998255           0.998258     
70-30 Decision Tree       0.998836            0.998854            0.998836           0.998837            0.998836            0.998854            0.998836           0.998837     
70-30 Random Forest       0.998836            0.998854            0.998836           0.998837            0.998836            0.998854            0.998836           0.998837     
60-40 Decision Tree       0.998255            0.998275            0.998255           0.998258            0.998

**Analisis dan Identifikasi kelemahan model**
=============================================
1. Akurasi Sebelum & Sesudah Tuning
  - Tidak ada perubahan yang signifikan setelah hyperparameter tuning.

  - Model sudah memiliki performa yang sangat tinggi sebelum tuning, sehingga tuning tidak banyak mempengaruhi hasil.

2. Precision & Recall
  - Precision dan Recall tetap stabil sebelum dan sesudah tuning.

  - Ini menunjukkan bahwa model dapat mengenali kelas-kelas dengan baik dan tidak mengalami ketidakseimbangan performa antar kelas.

3. Overfitting atau Underfitting?
  - Kemungkinan Overfitting:

    - Akurasi yang sangat tinggi (hampir 100%) bisa menjadi indikasi overfitting.

    - Jika model diuji dengan data baru yang berbeda dari data pelatihan, bisa jadi performanya menurun.

  - Tidak ada tanda underfitting:

    - Semua metrik menunjukkan performa tinggi, menandakan model tidak mengalami kesulitan dalam mempelajari pola data.


**Berikut Rekomendasi Tindakan Lanjutan**
============================================


1. Eksperimen dengan Algoritma Lain

  - Coba XGBoost atau Gradient Boosting untuk melihat apakah ada peningkatan performa.

  - Bisa juga mencoba Neural Networks jika dataset besar dan kompleks.

2. Kurangi Kompleksitas Model jika Perlu

  - Untuk Decision Tree, coba kurangi max_depth untuk menghindari overfitting.

  - Untuk Random Forest, bisa mengurangi jumlah n_estimators untuk efisiensi tanpa kehilangan performa.

3. Uji Model dengan Data Baru (Unseen Data)

  - Evaluasi model menggunakan dataset baru atau real-world data untuk memastikan model tidak overfitting.

4. Cek Data Imbalance

  - Jika dataset memiliki kelas yang tidak seimbang, pertimbangkan resampling (oversampling/undersampling) atau menggunakan class weight.